In [47]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import geopandas as gpd
from scipy.stats import norm
from __future__ import print_function, division

# Downloads

In [3]:
#download json y abrir. Ejemplo con plot
import requests
import json

url = 'https://s3.amazonaws.com/sb-public/sbg389_matplotlibrc.json' 
resp = requests.get(url = url)
data = json.loads(resp.text)

plt.rcParams.update(data)


/home/pipe/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/pipe/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# RESOLVER UNZIP

In [13]:
os.system("curl -O https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/mn_mappluto_16v1.zip")

# Unzip the files and move to the MNMapPluto subfolder (they all start with MN)
#os.system("unzip -jn mn_mappluto_16v1.zip MN*")
#os.system("mv " + "MN* " + os.getenv("PUIDATA") + "/MNMapPluto")

# Read the Manhattan SHP file that was obtained from the PLUTO dataset


0

In [15]:
bsize = gpd.read_file(os.getenv('PUIDATA') + "/mn_mappluto_16v1/MNMapPLUTO.shp")

bsize.head(2)

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.000000e+00,None,1592 2 AVENUE,C1-9/TA,None,2,468000.0,1435950.0,1.015450e+09,10885,...,0,1920,None,10028,C1-9,None,None,None,9a,"POLYGON ((997277.2344000041 221816.0936000049,..."
1,1.007230e+09,11/30/2006,263 9 AVENUE,C1-5/R8,None,2,539984.0,11879993.0,1.007238e+09,89203,...,0,1914,None,10001,R8,None,None,None,8d,"POLYGON ((984164.5626000017 211846.0703999996,..."


In [36]:
nrg = pd.read_csv("https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv")

In [37]:
nrg.columns

Index([u'Record Number', u'NYC Borough, Block, and Lot (BBL)',
       u'Co-reported BBL Status', u'BBLs Co-reported',
       u'Reported NYC Building Identificaiton Numbers (BINs)',
       u'Street Number', u'Street Name', u'Borough', u'Zip Code',
       u'DOF Benchmarking Submission Status', u'Site EUI(kBtu/ft2)',
       u'Weather Normalized Site EUI(kBtu/ft2)', u'Source EUI(kBtu/ft2)',
       u'Weather Normalized Source EUI(kBtu/ft2)',
       u'Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)',
       u'Automatic Water Benchmarking Eligible', u'Reported Water Method',
       u'ENERGY STAR Score', u'Total GHG Emissions(MtCO2e)',
       u'Direct GHG Emissions(MtCO2e)', u'Indirect GHG Emissions(MtCO2e)',
       u'Reported Property Floor Area (Building(s)) (ft²)',
       u'DOF Property Floor Area (Buildngs and Parking)(ft2)',
       u'Primary Property Type - Self Selected', u'DOF Number of Buildings'],
      dtype='object')

count               12204
unique               2168
top       See Primary BBL
freq                  657
Name: Site EUI(kBtu/ft2), dtype: object

In [43]:
#RENOMBRAR
nrg.rename(columns={'NYC Borough, Block, and Lot (BBL)':'BBL'}, inplace= True)


#TO NUMERIC, remplaza con NaNS el coerce
nrg['siteEUI'] = pd.to_numeric(nrg['Site EUI(kBtu/ft2)'], errors = 'coerce')
nrg['floorArea']= pd.to_numeric(nrg['Reported Property Floor Area (Building(s)) (ft²)'], errors = 'coerce')

#SELECT
nrg = nrg.copy().loc[:,['BBL','siteEUI','floorArea']]



In [44]:
nrg.siteEUI.describe()

count    1.154700e+04
mean     3.116677e+03
std      2.642014e+05
min      0.000000e+00
25%               NaN
50%               NaN
75%               NaN
max      2.822601e+07
Name: siteEUI, dtype: float64

In [45]:
#DROP NA, axis  0 borra filas, 1 columnas
#how, any con al menos 1, all tiene que ser todas
print (nrg.shape)
nrg.dropna(axis=0, how='any', thresh=None, inplace=True)
print (nrg.shape)

(16170, 3)
(11457, 3)


In [46]:
#SELECT BROADCASTING
#create a boolean array to use as mask
maks = nrg.

In [ ]:
#merge data sets by BBL keeping all rows from the energy data set
bblnrgdata = pd.merge(left = nrg, right=bsize, how = 'left', on = 'BBL')

In [50]:
#AREAS BAJO LA CURVA DE LA NOMAL

#Survival function 1 - cdf, but sf is sometimes more accurate). Area de la curva a la derecha del valor 
#sf(x, loc=0, scale=1), 1.65 es el z para 95%, esto me devuelve 5%
norm.sf(1.65, loc = 0, scale = 1,)

0.30853753872598688

In [55]:
#Percent point function (inverse of cdf — percentiles). Me va el valor de Z dada un area bajo la curva. 
#Area total, si quiero a dos colas, divido alpha por 2: 5% / 2 es 0.975 o 1 -  (alpha/2)
#http://images.slideplayer.es/17/5511464/slides/slide_10.jpg
norm.ppf(.95, loc = 0, scale = 1) 


1.6448536269514722